# Face Prediction with CNN(Keras)

Content:
* [Importing Analysis Libraries](#1)
* [Data Overview](#2)
* [Data Distribution](#3)
* [Image Shape](#4)
* [Reshape Data](#5)
* [Images Overview](#6)
* [Importing DL Libraries](#7)
* [Categorizing Target Values](#8)
* [Normalize Data](#9)
* [Image Data Generate](#10)
* [Train Test Split](#11)
* [Build Model](#12)
* [Callbacks](#13)
* [Ethnicity Prediction](#14)
* [Age Prediction](#15)
* [Gender Prediction](#16)

<a id="1"></a><br>
## Importing Analysis Libraries 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


<a id="2"></a><br>
## Data Overview

In [ ]:
data = pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
print(data.shape)
data.head()

In [ ]:
data = data.drop("img_name", axis=1)

In [ ]:
data.isnull().sum()

In [ ]:
columns = ["age","gender","ethnicity"]
y = data.drop("pixels", axis = 1)
x = data.drop(columns, axis = 1)

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
y.nunique()

<a id="3"></a><br>
## Data Distribution

In [ ]:
for i in y.columns :
    plt.figure(figsize=(15,7))
    g = sns.countplot(y[i], palette="icefire")
    plt.title("Number of {}".format(i))
    

In [ ]:
y["age"] = pd.cut(y["age"],bins=[0,3,18,45,64,116],labels=["0","1","2","3","4"])
plt.figure(figsize=(15,7))
g = sns.countplot(y["age"], palette="icefire")
plt.title("Number of age")

<a id="4"></a><br>
## Image Shape

In [ ]:
num_pixels = len(x['pixels'][0].split(" "))
img_height = int(np.sqrt(len(x['pixels'][0].split(" "))))
img_width = int(np.sqrt(len(x['pixels'][0].split(" "))))
print(num_pixels, img_height, img_width)

<a id="5"></a><br>
## Reshape Data

In [ ]:
x = pd.Series(x["pixels"])
x = x.apply(lambda x: x.split(' '))
x = x.apply(lambda x: np.array(list(map(lambda z: np.int(z), x))))
x = np.array(x)
x = np.stack(np.array(x), axis=0)
x = np.reshape(x, (-1, 48, 48))

# reshape data
x = x.reshape(-1,48,48,1)
print("x shape: ",x.shape)

<a id="6"></a><br>
## Images Overview

In [ ]:
plt.figure(figsize=(16,16))
for i,a in zip(np.random.randint(0, 23705, 25),range(1,26)):
    plt.subplot(5,5,a)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x[i])
    plt.xlabel(
        "Age:"+str(y['age'].iloc[i])+
        "  Ethnicity:"+str(y['ethnicity'].iloc[i])+
        "  Gender:"+ str(y['gender'].iloc[i])
    )
plt.show()

<a id="7"></a><br>
## Importing DL Libraries

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, InputLayer
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy

<a id="8"></a><br>
## Categorizing Target Values

In [ ]:
age_matrix = np.array(y["age"]) 
gender_matrix = np.array(y["gender"])
ethnicity_matrix = np.array(y["ethnicity"])
age = to_categorical(age_matrix, num_classes = 5)
gender = to_categorical(y["gender"], num_classes = 2)
ethnicity = to_categorical(ethnicity_matrix, num_classes = 5)
print(age, gender, ethnicity)

<a id="9"></a><br>
## Normalize Data

In [ ]:
# Normalize the data
x = x / 255.0
x = x / 255.0

<a id="10"></a><br>
## Image Data Generate

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x)

<a id="11"></a><br>
## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train_ethnicity, X_test_ethnicity, y_train_ethnicity, y_test_ethnicity = train_test_split(x, ethnicity, test_size= 0.3)
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(x, gender, test_size= 0.3)
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(x, age, test_size= 0.3)

<a id="12"></a><br>
## Build Model

In [ ]:
def my_model(num_classes, activation, loss):
    model = Sequential() 
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding = "same", input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding = "same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding = "same"))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation= activation))
    model.compile(optimizer='Adam',
              loss= loss,
              metrics=['accuracy'])
    return model

<a id="13"></a><br>
## Callbacks

In [ ]:
early_stopping = EarlyStopping(patience=10, min_delta=0.001,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, 
                                            factor=0.5, min_lr=0.00001)

<a id="14"></a><br>
## Ethnicity Prediction

In [ ]:
epochs = 200  # for better result increase the epochs
batch_size = 64
model_ethnicity = my_model(5,"softmax",'categorical_crossentropy')
history_ethnicity = model_ethnicity.fit(X_train_ethnicity, y_train_ethnicity, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_ethnicity,y_test_ethnicity), steps_per_epoch= X_train_ethnicity.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

In [ ]:
fig = px.line(
    history_ethnicity.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
loss, acc = model_ethnicity.evaluate(X_test_ethnicity, y_test_ethnicity, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

<a id="15"></a><br>
## Age Prediction

In [ ]:
model_age = my_model(5,"softmax",'categorical_crossentropy')
history_age = model_age.fit(X_train_age, y_train_age, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_age,y_test_age), steps_per_epoch= X_train_age.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

In [ ]:
fig = px.line(
    history_age.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
loss, acc = model_age.evaluate(X_test_age, y_test_age, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

<a id="16"></a><br>
## Gender Prediction

In [ ]:
model_gender = my_model(2, "sigmoid", "binary_crossentropy")
history_gender = model_gender.fit(X_train_gender, y_train_gender, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test_gender,y_test_gender), steps_per_epoch= X_train_gender.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

In [ ]:
fig = px.line(
    history_gender.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
loss, acc = model_gender.evaluate(X_test_gender, y_test_gender, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

 If you like my notebok, please don't forget to upvote. I would be happy to see your thoughts and suggestions about my notebook in the comments section. I hope everything goes the way you want. Have a nice work!!